In [9]:
import tensorflow as tf
import math
import timeit
from datetime import datetime

In [10]:
def feed_forward_func(x, y):
    sigy = tf.sigmoid(y)#sigmoid function with respect to y    #(1)
    num = x+sigy# numerator completed                               #(2)
    sigx = tf.sigmoid(x)#sigmoid function with respect to x    #(3)
    xpy = x+y                                                       #(4)
    xpysqr = xpy**2                                                 #(5)
    den = sigx + xpysqr #denominator                                #(6)
    invden = 1/ den                                                 #(7)
    f  = num * invden
    return f

In [11]:
tf_feed_forward_func = tf.function(feed_forward_func)

In [12]:
A = tf.Variable(3.0)
B = tf.Variable(-4.0)
tf_feed_forward_func(A, B)

2022-05-12 14:18:04.725038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(), dtype=float32, numpy=1.5456449>

In [13]:
tf.autograph.to_code(feed_forward_func)

"def tf__feed_forward_func(x, y):\n    with ag__.FunctionScope('feed_forward_func', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:\n        do_return = False\n        retval_ = ag__.UndefinedReturnValue()\n        sigy = ag__.converted_call(ag__.ld(tf).sigmoid, (ag__.ld(y),), None, fscope)\n        num = ag__.ld(x) + ag__.ld(sigy)\n        sigx = ag__.converted_call(ag__.ld(tf).sigmoid, (ag__.ld(x),), None, fscope)\n        xpy = ag__.ld(x) + ag__.ld(y)\n        xpysqr = ag__.ld(xpy) ** 2\n        den = ag__.ld(sigx) + ag__.ld(xpysqr)\n        invden = 1 / ag__.ld(den)\n        f = ag__.ld(num) * ag__.ld(invden)\n        try:\n            do_return = True\n            retval_ = ag__.ld(f)\n        except:\n            do_return = False\n            raise\n        return fscope.ret(retval_, do_return)\n"

In [14]:
tf_feed_forward_func.get_concrete_function(tf.constant(1.0), tf.constant(1.0)).graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "y"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "y"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "Sigmoid"
  op: "Sigmoid"
  input: "y"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "add"
  op: "AddV2"
  input: "x"
  input: "Sigmoid"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "Sigmoid_1"
  op: "Sigmoid"
  input: "x"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "add_1"
  op: "AddV2"
  input: "x"
  input: "y"
  attr {
    key: "T"
    value {
      type: D

In [27]:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.random.normal([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [28]:
class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model!
my_model = SequentialModule(name="the_model")

In [30]:
# Call it, with random results
print(my_model(tf.constant([[2.0, 2.0, 2.0]])))
my_model.variables

tf.Tensor([[2.4688256 2.5380783]], shape=(1, 2), dtype=float32)


(<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([-0.11463245, -1.6770751 , -0.16329676], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.41852248,  1.2297032 , -0.34434792],
        [ 0.9649756 ,  1.1143554 ,  1.0111719 ],
        [ 1.2907805 ,  0.6392945 , -0.3404188 ]], dtype=float32)>,
 <tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0.2691343 , 0.10299923], dtype=float32)>,
 <tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.45411962, -0.644501  ],
        [ 0.24688417,  1.1109371 ],
        [-0.97227424, -0.07377498]], dtype=float32)>)